In [18]:
import torch
from torch.utils.tensorboard import SummaryWriter
import csv
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import os
from statistics import mean
from numpy.linalg import norm
import numpy as np

In [3]:
# Design prompt 3, human prompts
human = []
with open("data/amazonTurkDesPrompt4.csv", 'r') as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        human.append(row[0])
human = [i.replace('\n',' ') for i in human]    # Removing the line breaks as they mess with tensorboard.
human

['ï»¿A vest made of crystal quartz pieces connected by thin copper infused string. The vest is then draped over a machine after being charged for 8-24 hrs by human wear.',
 'Perhaps there could be something...a cord?...attached to a bicycle or treadmill that would carry the energy to a port of some sort. The port would collect and convert the human energy for use elsewhere.',
 'a device which is attached to the joints so when they move they turn a coil which generates energy.',
 'When you blush, you put sensors on your cheeks and the warmth transfers to energy. This is effective for embarrassment, excitement, and joy.',
 'People could wear kinetic bracelets and anklets that could collect and store energy from normal human activities like walking. Pumping your arms while walking would collect additional energy. The energy could be downloaded by removing the bracelets and anklets and placing them on a reverse charging pad, which could then power other devices.',
 'solar equipment collect

In [4]:
# Design prompt 3, gpt prompts
desprompt4 = pd.read_csv('data/DesPrompt4_100Soln_FP0.csv', sep = ",")
lst = []
for d in desprompt4.values:
    lst.append(d[0])
gpt_FP0 = []
for i in range(len(lst)):
    mod_statements = lst[i].replace(lst[i][:3], '')
    mod_statements = mod_statements.strip()
    gpt_FP0.append(mod_statements)
gpt_FP0[98] = gpt_FP0[98].replace('. ', '')


In [5]:
desprompt4_FP0_PP2 = pd.read_csv('data/DesPrompt4_100Soln_FP0_PP2.csv', sep = ",")
lst = []
for d in desprompt4_FP0_PP2.values:
    lst.append(d[0])
gpt_FP0_PP2 = []
for i in range(len(lst)):
    mod_statements = lst[i].replace(lst[i][:3], '')
    mod_statements = mod_statements.strip()
    gpt_FP0_PP2.append(mod_statements)
gpt_FP0_PP2[98] = gpt_FP0_PP2[98].replace('. ', '')

In [6]:
desprompt4_FP1 = pd.read_csv('data/DesPrompt4_100Soln_FP1.csv', sep = ",")
lst = []
for d in desprompt4_FP1.values:
    lst.append(d[0])
gpt_FP1 = []
for i in range(len(lst)):
    mod_statements = lst[i].replace(lst[i][:3], '')
    mod_statements = mod_statements.strip()
    if mod_statements[0] == ')':
        mod_statements = mod_statements.replace(')', '')
    if mod_statements[0] == '.':
        mod_statements = mod_statements.replace('.', '')
    if mod_statements[0:2] == '4 ':
        mod_statements = mod_statements.replace('4 ', '')
    gpt_FP1.append(mod_statements)
gpt_FP1

['An ankle bracelet with a wind-up mechanism that generates power when walking or running.',
 'Foot pedals connected to a dynamo that produces energy while walking or jogging on treadmills or stairmasters.',
 'A treadmill designed to harvest kinetic energy with every stride and store it in rechargeable batteries for later use.',
 'Wristbands fitted with mini-generators which produce small amounts of electricity when moved up & down or rotated while typing, playing video games, etcetera..',
 'A device featured on treadmills, stationary bikes and ellipticals which captures kinetic energy produced by cycling, running and walking respectively and stores it for further use as electrical power in rechargeable batteries connected directly to those devices .',
 "Electronic cubicles installed at gyms where users generate electrical power by pedaling on ergometers (ergometers generate electric current due to their varying resistance against the user's motion).",
 'Robotic exoskeleton suits that 

In [7]:
# embed data
model = SentenceTransformer('all-MiniLM-L6-v2')
gpt_FP0_embeddings = model.encode(gpt_FP0)
gpt_FP0_PP2_embeddings = model.encode(gpt_FP0_PP2)
gpt_FP1_embeddings = model.encode(gpt_FP1)
human_embeddings = model.encode(human)

In [8]:
bert_output_human = torch.from_numpy(human_embeddings)
bert_output_gpt_FP0 = torch.from_numpy(gpt_FP0_embeddings)
bert_output_gpt_FP0_PP2 = torch.from_numpy(gpt_FP0_PP2_embeddings)
bert_output_gpt_FP1 = torch.from_numpy(gpt_FP1_embeddings)

In [9]:
bert_output_gpt_FP1

tensor([[-0.1310,  0.0865, -0.0136,  ..., -0.0233, -0.0274,  0.0179],
        [-0.0891,  0.0217, -0.0144,  ...,  0.0011,  0.0267, -0.0334],
        [-0.1362,  0.0422, -0.0383,  ..., -0.0186,  0.0215,  0.0011],
        ...,
        [-0.0744,  0.0209, -0.0141,  ..., -0.0511,  0.0312, -0.0275],
        [-0.0950,  0.0184,  0.0395,  ..., -0.1207,  0.0055, -0.0402],
        [ 0.0351,  0.0526, -0.0031,  ..., -0.0094, -0.0863,  0.0572]])

In [10]:
metadata = [('human', line) for line in human] + [('gpt_FP0', line) for line in gpt_FP0] + [('gpt_FP0_PP2', line) for line in gpt_FP0_PP2] + [('gpt_FP1', line) for line in gpt_FP1]
embeddings = torch.cat((bert_output_human, bert_output_gpt_FP0, bert_output_gpt_FP0_PP2, bert_output_gpt_FP1), 0)

In [28]:
# visualize embeddings
writer = SummaryWriter('embeddings_4')
writer.add_embedding(embeddings, metadata=metadata, metadata_header=['writer', 'prompt'])

Run in terminal: tensorboard --logdir="embeddings\" --host localhost http://localhost:6006/ to open Tensorboard Refresh page.
Note when you run from anaconda, conda activate chatgpt -> cd chatgpt -> run the terminal command above.

## Save Embeddings

In [15]:
def save_embeddings(data, metadata, file_name=None):
    if file_name is None:
        dest = '/data/embeddings/'
    else:
        dest = os.getcwd() + '/data/embeddings/' + file_name + '.pkl'
    os.makedirs('data/embeddings/', exist_ok=True)

    writer = [x[0] for x in metadata]
    prompt = [x[1] for x in metadata]
    embedding_df = pd.DataFrame(list(zip(writer, prompt, data)), columns=["writer", "prompt", "embedding"])

    embedding_df.to_pickle(dest)

    return embedding_df

In [16]:
file_name = 'prompt4_embeddings'
embedding_df = save_embeddings(embeddings, metadata, file_name=file_name)

## Load embeddings

In [14]:
# TODO


# Similarity Metrics
## Nearest Generated Sample
Using cosine distance

In [28]:
writers = ['gpt_FP0', 'gpt_FP0_PP2', 'gpt_FP1']

original_datapoints = embedding_df[embedding_df['writer'] == 'human']['embedding'].tolist()
for writer in writers:
    generated_datapoints = embedding_df[embedding_df['writer'] == writer]['embedding'].tolist()
    distances = []
    for original_datapoint in original_datapoints:
        min_distance = 1e9
        for generated_datapoint in generated_datapoints:
            distance = np.dot(original_datapoint, generated_datapoint)/(norm(original_datapoint)*norm(generated_datapoint))
            if distance < min_distance:
                min_distance = distance
        distances.append(min_distance)
    mean_distance = mean(distances)
    print(f"Mean of nearest generated samples for {writer}: {mean_distance}")


Mean of nearest generated samples for gpt_FP0: 0.29521146416664124
Mean of nearest generated samples for gpt_FP0_PP2: 0.03285186365246773
Mean of nearest generated samples for gpt_FP1: 0.04373559355735779
